# Home Credit Modeling

In [65]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import iqr, randint, uniform
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from skopt.plots import plot_convergence, plot_objective
import pickle
import gc

%load_ext autotime

gc.enable()
np.random.seed(235)

path = "/Users/dsaxton/home_credit_default/"

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.06 ms


#### Load the data

In [117]:
df = pd.read_csv(path + "train.csv")

time: 22.6 s


In [118]:
sk_id_curr = df.pop("SK_ID_CURR")
y = df.pop("TARGET")

time: 438 ms


#### Select features

In [119]:
clf = lgb.LGBMClassifier(n_estimators=1000, num_leaves=23, subsample=0.5)
clf.fit(df, y)

lgb_cols = df.columns[clf.feature_importances_ > 0]
len(lgb_cols)

456

time: 1min 46s


In [120]:
var_imp = pd.DataFrame({"Feature": df.columns, "Importance": clf.feature_importances_})[["Feature", "Importance"]].sort_values("Importance", ascending=False)

time: 6.01 ms


#### View important features

In [124]:
var_imp.head()

,Feature,Importance
31,EXT_SOURCE_2,368
32,EXT_SOURCE_3,350
359,AMT_CREDIT_DIV_AMT_ANNUITY,288
399,EXT_SOURCE_1_DIV_DAYS_BIRTH,222
8,DAYS_BIRTH,205


time: 6.43 ms


#### Average importance over non-zero importance features

In [122]:
int(np.nanmean(var_imp["Importance"].where(var_imp["Importance"] > 0)))

48

time: 5.01 ms


#### Importance of selected features

In [129]:
features = ["MAX_CREDIT_CARD_SK_DPD_12M"]

var_imp[var_imp["Feature"].isin(features)]

,Feature,Importance
274,MAX_CREDIT_CARD_SK_DPD_12M,3


time: 14.6 ms


#### LightGBM CV

In [ ]:
params = {"n_estimators": 20000, 
          "num_leaves": 350, 
          "min_data_in_leaf": 1000,
          "learning_rate": 0.005, 
          "bagging_fraction": 0.5, 
          "feature_fraction": 0.5, 
          "lambda_l2": 1}

lgb_data = lgb.Dataset(data=df[lgb_cols], 
                       label=y)

cv_result = lgb.cv(params=params, 
                   train_set=lgb_data, 
                   nfold=5, 
                   metrics="auc", 
                   early_stopping_rounds=200, 
                   stratified=True, 
                   shuffle=True, 
                   verbose_eval=100, 
                   show_stdv=True, 
                   seed=2357)

cv_result = pd.DataFrame(cv_result)

/Users/dsaxton/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	cv_agg's auc: 0.76644 + 0.00260544
[200]	cv_agg's auc: 0.770319 + 0.00260403
[300]	cv_agg's auc: 0.773589 + 0.00235572
[400]	cv_agg's auc: 0.77651 + 0.00222441
[500]	cv_agg's auc: 0.779094 + 0.00212385


In [ ]:
cv_result.tail()

#### Fit chosen model

CV mean:  0.791015

In [78]:
lgb_model = lgb.LGBMClassifier(n_estimators=2520, 
                         num_leaves=233, 
                         min_data_in_leaf=1000,
                         learning_rate=0.005, 
                         bagging_fraction=0.5, 
                         feature_fraction=0.5, 
                         lambda_l2=0.8)

lgb_model.fit(df[lgb_cols], y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
        learning_rate=0.0025, max_depth=10, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=6876,
        n_jobs=-1, num_leaves=113, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.8, silent=True, subsample=0.5,
        subsample_for_bin=200000, subsample_freq=1)

time: 30min 31s


#### Score test data and generate submission

In [79]:
df = pd.read_csv(path + "test.csv")

time: 4.26 s


In [80]:
df["NAME_INCOME_TYPE_Maternity_leave"] = 0

time: 10 ms


In [81]:
submission = pd.DataFrame({"SK_ID_CURR": df["SK_ID_CURR"], 
                           "TARGET": lgb_model.predict_proba(df[lgb_cols])[:,1]})

submission.head()

,SK_ID_CURR,TARGET
0,100001,0.033505
1,100005,0.123829
2,100013,0.026391
3,100028,0.033826
4,100038,0.170974


time: 26.3 s


In [82]:
submission.to_csv(path + "submission.csv", index=False)

time: 176 ms
